# 📘 Preprocesamiento del Dataset IMDB

En este notebook realizamos el **preprocesamiento de texto** para el modelo de clasificación de sentimiento de reseñas de películas IMDb. Esta etapa incluye:

1. Cargar datos limpios desde CSV
2. Procesar etiquetas
3. Tokenizar texto (vectorización)
4. Aplicar padding
5. Guardar arrays procesados para entrenamiento


# 📘 Preprocesamiento del Dataset IMDB

En este paso realizamos la tokenización, padding y codificación de etiquetas para preparar los datos para entrenamiento. También guardamos los resultados para su reutilización.


In [2]:
# 1. Importar librerías
import pandas as pd
import numpy as np
import re
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


2025-07-17 16:39:27.261749: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-17 16:39:28.038709: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-17 16:39:30.205117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## 📥 Cargar datos limpios

Utilizamos el archivo `clean_imdb_100k.csv`, que contiene reseñas ya limpiadas (sin HTML, puntuación ni stopwords) y con su respectiva etiqueta.


In [3]:
# 2. Cargar el dataset limpio desde CSV
df = pd.read_csv("../data/clean_imdb_100k.csv")
print(df.head())


                                          clean_text  label
0  run do not look back i made the mistake of buy...      1
1  excellent book with pictures for teens to lear...      2
2  improving your quality of relationships maxwel...      2
3  no price no need to buy if amazon is unwilling...      1
4  great toy holder this is an awesome product fo...      2


3. 🔹 Preparar etiquetas (opcional: convertir a 0 y 1)

In [4]:
labels = df["label"] - 1  # Asegúrate que la etiqueta original era 1 y 2


4. 🔹 Tokenización y conversión a secuencias

In [5]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df["clean_text"])

sequences = tokenizer.texts_to_sequences(df["clean_text"])

5. 🔹 Padding de secuencias

In [6]:
padded = pad_sequences(sequences, maxlen=200, padding="post", truncating="post")
print("✅ Padding listo. Shape:", padded.shape)

✅ Padding listo. Shape: (100000, 200)


6. 🔹 Guardar datos procesados

In [7]:
np.savez_compressed("../data/processed_data.npz", X=padded, y=labels)
print("✅ Datos guardados en ../data/processed_data.npz")


✅ Datos guardados en ../data/processed_data.npz


7. 🔹 Guardar el tokenizer

In [8]:
with open("../data/tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
print("✅ Tokenizer guardado en ../data/tokenizer.pkl")

✅ Tokenizer guardado en ../data/tokenizer.pkl


In [9]:
from sklearn.model_selection import train_test_split

# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(padded, labels, test_size=0.2, random_state=42)

# Guardar conjuntos de datos
np.savez_compressed("../data/processed_split.npz", 
                    X_train=X_train, X_val=X_val, 
                    y_train=y_train, y_val=y_val)

print("✅ Datos divididos y guardados correctamente.")

✅ Datos divididos y guardados correctamente.


## 📊 Estadísticas del corpus

Aquí mostramos estadísticas básicas del texto procesado y el tokenizer:

In [10]:
# Estadísticas útiles
sequence_lengths = [len(seq) for seq in sequences]
print("✅ Número de secuencias:", len(sequences))
print("✅ Longitud promedio de secuencia:", np.mean(sequence_lengths))
print("✅ Longitud máxima:", np.max(sequence_lengths))
print("✅ Tamaño del vocabulario:", len(tokenizer.word_index))

✅ Número de secuencias: 100000
✅ Longitud promedio de secuencia: 77.87445
✅ Longitud máxima: 216
✅ Tamaño del vocabulario: 190637


## 💾 Guardar arrays procesados

Guardamos los datos como archivo `.npz` para usarlos en el notebook de entrenamiento (`03_train_model.ipynb`) y también exportamos el CSV limpio.
